In [ ]:
%pylab inline
import scipy
import h5py
import skimage
from skimage import io,transform 
from collections import OrderedDict
# old_code_dir = home + '/caffe_projects/TextureModel/code/'
# sys.path.append(old_code_dir)
# from DeepImageSynthesis import *
home = '/gpfs01/bethge/home/lgatys/'
project_dir = home + 'NeuralImageSynthesis/'
photo_dir = home + '/neural-style/images/photos/'
art_dir = home + '/neural-style/images/paintings/'

In [ ]:
def make_torch_input(filename, layers, loss_functions, args):
    f = h5py.File(filename,'w')
    for l,layer in enumerate(layers):
        layer_group = f.create_group(layer)
        for lf,loss_function in enumerate(loss_functions[l]):
            lf_group = layer_group.create_group(loss_function)
            for arg in args[l][lf]:
                dataset = lf_group.create_dataset(arg, data=args[l][lf][arg])
    f.close()
    
def make_torch_init(filename, init):
    f = h5py.File(filename,'w')
    f.create_dataset('init', data=init)
    f.close()

def get_torch_output(filename):
    f = h5py.File(filename,'r')
    data = f['opt_result']
    return data.value
    f.close()

def pass_layers(layers):
    '''
    Takes list of layers and returns comma separated list
    '''
    csl = str()
    for l in layers:
        csl = csl+l+','
    return csl[:-1]

def get_activations(images, caffe_model, layers='all', gpu=1):
    '''
    Function to get neural network activations in response to images from torch.
    
    :param images: array of images
    :param caffe_model: file name of the network .caffemodel file
    :param layers: network layers for which the activations should be computed
    :return: network activations in response to images
    '''
    layers = pass_layers(layers)
    tmp_dir = project_dir + 'Tmp/'
    images_file_name = tmp_dir + 'images.hdf5'
    output_file_name = tmp_dir + 'activations.hdf5'
    f = h5py.File(images_file_name, 'w')
    f.create_dataset('images', data=images)
    f.close()
    context = {
    'caffe_model': caffe_model,
    'images': images_file_name,
    'layers': layers,
    'gpu': gpu,
    'backend': 'cudnn',
    'output_file': output_file_name
    }

    template = ('#!/bin/bash\n' +
                '/usr/local/torch/install/bin/th ComputeActivations.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-images {images} ' + 
                '-layers {layers} ' + 
                '-gpu {gpu} ' + 
                '-backend {backend} ' +
                '-output_file {output_file}')

    script_name = project_dir + 'get_activations.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    #execute script
    !cd /gpfs01/bethge/home/lgatys/NeuralImageSynthesis/ && \
    ./get_activations.sh >/dev/null
    
    f = h5py.File(output_file_name,'r')
    act = OrderedDict()
    for key in f.keys():
        act[key] = f[key].value.copy()
    f.close()
    return act

def preprocess(image):
    assert(image.max() <= 1.)
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image_torch = 255 * (image[:,:,::-1] - imagenet_mean).transpose(2,0,1)
    return image_torch

def deprocess(image_torch):
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image = (image_torch.transpose(1,2,0)/255. + imagenet_mean)[:,:,::-1]
    image[image>1] = 1
    image[image<0] = 0
    return image

def gram_matrix(activations):
    n_fm = activations.shape[0]
    F = activations.reshape(n_fm,-1)
    G = F.dot(F.T) / F.size
    return G

In [ ]:
#get images
img_size = 450.
conditions = ['content','style']
img_names = OrderedDict()
img_names['content'] = 'leon_cropped.jpg'
img_names['style'] = 'vangogh_selfportrait.jpg'
imgs = OrderedDict()
imgs['content'] = imread(photo_dir + img_names['content'])
imgs['content'] = transform.pyramid_reduce(imgs['content'], sqrt(float(imgs['content'][:,:,0].size) / img_size**2))
imgs['style'] = imread(art_dir + img_names['style'])
imgs['style'] = transform.pyramid_reduce(imgs['style'], sqrt(float(imgs['style'][:,:,0].size) / img_size**2))
imgs_torch = OrderedDict()
for cond in conditions:
    imshow(imgs[cond]);show()
    imgs_torch[cond] = preprocess(imgs[cond])

In [ ]:
#get activations
caffe_model = home + '/neural-style/models/VGG_ILSVRC_19_layers_conv.caffemodel'
act = OrderedDict()
for cond in conditions:
    act[cond] = get_activations(imgs_torch[cond], caffe_model,
                                           layers=['relu1_1','relu2_1','relu3_1','relu4_1','relu4_2','relu5_1'])

In [ ]:
input_file_name = project_dir + 'Tmp/input.hdf5'
init_file_name = project_dir + 'Tmp/init.hdf5'
output_file_name = project_dir + 'Results/TorchOutput/output.hdf'
layers = [
    'relu1_1',
    'relu2_1',
    'relu3_1',
    'relu4_1',
    'relu4_2',
    'relu5_1'
]
loss_functions = [
    ['GramMSE'],
    ['GramMSE'],
    ['GramMSE'],
    ['GramMSE'],
    ['MSE'],
    ['GramMSE']
]
args = [
    [
        {'target': gram_matrix(act['style']['relu1_1']),
          'weight': array([1e3]), 'norm': array([0])}
    ],
        [
        {'target': gram_matrix(act['style']['relu2_1']),
          'weight': array([1e3]), 'norm': array([0])}
    ],
        [
        {'target': gram_matrix(act['style']['relu3_1']),
          'weight': array([1e3]), 'norm': array([0])}
    ],
        [
        {'target': gram_matrix(act['style']['relu4_1']),
          'weight': array([1e3]), 'norm': array([0])}
    ],
        [
        {'target': act['content']['relu4_2'],
          'weight': array([1]), 'norm': array([0])}
    ],
        [{'target': gram_matrix(act['style']['relu5_1']),
          'weight': array([1e3]), 'norm': array([0])}
        ]
]
make_torch_input(input_file_name, layers, loss_functions, args)
make_torch_init(init_file_name, imgs_torch['content'])

In [ ]:
gpu = 1
max_iter = 1000
context = {
    'caffe_model': caffe_model,
    'input_file': input_file_name,
    'init_file': init_file_name,
    'gpu': gpu,
    'max_iter': max_iter,
    'backend': 'cudnn',
    'print_iter': 50,
    'save_iter': 0,
    'layer_order': pass_layers(layers),
    'output_file': output_file_name
}

template = (
            '#!/bin/bash\n' +
            'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
            '-caffe_model {caffe_model} ' +
            '-input_file {input_file} ' + 
            '-init_file {init_file} ' + 
            '-gpu {gpu} ' + 
            '-max_iter {max_iter} ' +
            '-print_iter {print_iter} ' +
            '-save_iter {save_iter} ' +
            '-backend {backend} ' + 
            '-layer_order {layer_order} ' +
            '-output_file {output_file}'
           )

script_name = project_dir + 'run_synthesis.sh'
with open(script_name, 'w') as script:
    script.write(template.format(**context))
#execute script
!cd /gpfs01/bethge/home/lgatys/NeuralImageSynthesis/ && \
./run_synthesis.sh

In [ ]:
output = deprocess(get_torch_output(output_file_name))

In [ ]:
imshow(output)

In [ ]:
imsave(project_dir + 'Results/Images/test3.jpg', output)